In [1]:
from typing import List, Optional, Dict, Any, Tuple
import os
from pathlib import Path
import copy
import time 
from collections import defaultdict

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import pickle
import albumentations as A
from albumentations.pytorch import ToTensorV2
import gc
import json
from tqdm import tqdm
try:
    import wandb
except Exception:
    !pip install --upgrade wandb
    import wandb
try:
    import timm
except Exception:
    !pip install --upgrade timm
    import timm

pd.options.display.float_format = '{: ,.4f}'.format
pd.options.display.max_colwidth = 0

try:
    from helpers import pandas_utils as pu, files as f, visualization as vis
    # raise
except Exception:
    !pip install -q -I git+http://github.com/PaulDanielML/common_utils.git
    from helpers import pandas_utils as pu, files as f, visualization as vis

In [2]:
CONFIG = {
    "seed": 319,
    "epochs": 5,
    "img_size": 448,
    "args_hor_flip": {"p": 0.5},
    "args_ver_flip": {"p": 0.5},
    "args_rot": {"p": 0.5, "limit": 30},
    "model_name": "tf_efficientnet_b0",
    "num_classes": 15587,
    "train_batch_size": 32,
    "valid_batch_size": 64,
    "learning_rate": 1e-4,
    "scheduler": "CosineAnnealingLR",
    "min_lr": 1e-6,
    "T_max": 500,
    "weight_decay": 1e-6,
    "n_fold": 5,
    "n_accumulate": 1,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    # ArcFace Hyperparameters
    "s": 30.0,
    "m": 0.50,
    "ls_eps": 0.0,
    "easy_margin": False,
}


def make_transforms(config: Dict):
    param_mapping = {
        "args_hor_flip": A.HorizontalFlip,
        "args_ver_flip": A.VerticalFlip,
        "args_rot": A.Rotate,
    }

    base_transforms = [
        A.Resize(config["img_size"], config["img_size"]),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
        ToTensorV2(),
    ]

    train_transforms = copy.deepcopy(base_transforms)

    for arg_name, arg_class in param_mapping.items():
        if config[arg_name]["p"] > 0.0:
            train_transforms.insert(1, arg_class(**config[arg_name]))

    return {"train": A.Compose(train_transforms), "valid": A.Compose(base_transforms)}

transforms = make_transforms(CONFIG)


In [3]:
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [4]:
ON_KAGGLE_KERNEL = os.path.isdir("/kaggle/input")
print(ON_KAGGLE_KERNEL)
if ON_KAGGLE_KERNEL:
    DATA_DIR = Path("/kaggle/input/happy-whale-and-dolphin")
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        api_key = user_secrets.get_secret("wandb_api")
        wandb.login(key=api_key)
        wandb_active = True
    except:
        wandb_active = False
else:
    from src import DATA_DIR, SUB_DIR

False


In [5]:
def encode_labels(df: pd.DataFrame) -> pd.DataFrame:
    enc = LabelEncoder()
    if "individual_id" not in df.columns:
        return df
    df = df.copy(deep=True)
    print("Encoding labels of df...")
    df['individual_id'] = enc.fit_transform(df['individual_id'])
    # if not os.path.isfile("id_encoding.pickle"):
    print("Saving new label encoder...")
    with open("id_encoding.pickle", "wb") as f:
        pickle.dump(enc, f)

    return df

In [6]:
class WhaleDataset(Dataset):
    def __init__(
        self, df: pd.DataFrame, transforms: Optional[A.Compose] = None, labels: bool = True
    ):
        self.df = df.copy(deep=True)
        self.labels = labels
        self.transforms = transforms
        self.img_dir = DATA_DIR / "train_images" if labels else DATA_DIR / "test_images"
        self.df["img_path"] = self.df["image"].apply(lambda x: self.img_dir / x)
        if labels and (self.df["individual_id"].dtype == "object"):
            self.df = encode_labels(self.df)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img = np.array(Image.open(row.img_path).convert("RGB"))

        if self.transforms:
            img = self.transforms(image=img)["image"]

        data = {"image": img}
        if not self.labels:
            # return data.update({"img_name": [row.image]})
            return data
        return data.update({"label": torch.tensor(row.individual_id, dtype=torch.long)})


In [7]:
def create_folds(df: pd.DataFrame) -> pd.DataFrame:
    df_local = df.copy(deep=True)
    print("Creating folds...")
    skf = StratifiedKFold(5)
    for fold_number, (_, test_idx) in enumerate(skf.split(X=df_local, y=df_local.individual_id)):
        df_local.loc[test_idx, "fold_number"] = fold_number
    return df_local

In [8]:
def create_dataloaders(df: pd.DataFrame, fold: int = 0) -> Tuple[DataLoader, DataLoader]:
    df_train = df[df["fold_number"] != fold].reset_index(drop=True)
    df_valid = df[df["fold_number"] == fold].reset_index(drop=True)

    train_dataset = WhaleDataset(df_train, transforms=transforms["train"])
    valid_dataset = WhaleDataset(df_valid, transforms=transforms["valid"])

    train_loader = DataLoader(
        train_dataset,
        batch_size=CONFIG["train_batch_size"],
        num_workers=2,
        shuffle=True,
        pin_memory=True,
        drop_last=True,
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=CONFIG["valid_batch_size"],
        num_workers=2,
        shuffle=False,
        pin_memory=True,
    )

    return train_loader, valid_loader


In [9]:
model = timm.create_model(CONFIG["model_name"], pretrained=True, num_classes=CONFIG["num_classes"])
model.to(CONFIG['device']);
MODEL_TRAINED = False

In [10]:
def criterion(outputs, labels):
    return nn.CrossEntropyLoss()(outputs, labels)

In [11]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        images = data['image'].to(device, dtype=torch.float)
        labels = data['label'].to(device, dtype=torch.long)
        
        batch_size = images.size(0)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss = loss / CONFIG['n_accumulate']
            
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [12]:
@torch.inference_mode()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        images = data['image'].to(device, dtype=torch.float)
        labels = data['label'].to(device, dtype=torch.long)
        
        batch_size = images.size(0)

        outputs = model(images)
        loss = criterion(outputs, labels)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss)   
    
    gc.collect()
    
    return epoch_loss

In [13]:
def run_training(model, optimizer, scheduler, num_epochs):
    # To automatically log gradients
    wandb.watch(model, log_freq=100)

    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    history = defaultdict(list)

    for epoch in range(1, num_epochs + 1):
        gc.collect()
        train_epoch_loss = train_one_epoch(
            model,
            optimizer,
            scheduler,
            dataloader=train_loader,
            device=CONFIG["device"],
            epoch=epoch,
        )

        val_epoch_loss = valid_one_epoch(model, val_loader, device=CONFIG["device"], epoch=epoch)

        history["Train Loss"].append(train_epoch_loss)
        history["Valid Loss"].append(val_epoch_loss)

        # Log the metrics
        wandb.log({"Train Loss": train_epoch_loss})
        wandb.log({"Valid Loss": val_epoch_loss})

        # deep copy the model
        if val_epoch_loss <= best_epoch_loss:
            print(f"Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            run.summary["Best Loss"] = best_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "Loss{:.4f}_epoch{:.0f}.bin".format(best_epoch_loss, epoch)
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved.")

        print()

    end = time.time()
    time_elapsed = end - start
    print(
        "Training complete in {:.0f}h {:.0f}m {:.0f}s".format(
            time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60
        )
    )
    print("Best Loss: {:.4f}".format(best_epoch_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, history


In [14]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [15]:
df: pd.DataFrame = pd.read_csv(DATA_DIR / "train.csv")
df = encode_labels(df)
df = create_folds(df)
train_loader, val_loader = create_dataloaders(df)
optimizer = optim.Adam(
    model.parameters(), lr=CONFIG["learning_rate"], weight_decay=CONFIG["weight_decay"]
)
scheduler = fetch_scheduler(optimizer)

# run = wandb.init(
#     project="HappyWhale",
#     config=CONFIG,
#     job_type="Train",
#     tags=["Standard classifier", "efficientnet_b0", "448"],
#     anonymous="must",
# )

# model, history = run_training(
#     model, optimizer, scheduler, num_epochs=CONFIG["epochs"]
# )
# MODEL_TRAINED = True
# run.finish()


Encoding labels of df...
Saving new label encoder...
Creating folds...


In [16]:
df

,image,species,individual_id,fold_number
0,00021adfb725ed.jpg,melon_headed_whale,12348,0.0000
1,000562241d384d.jpg,humpback_whale,1636,1.0000
2,0007c33415ce37.jpg,false_killer_whale,5842,0.0000
3,0007d9bca26a99.jpg,bottlenose_dolphin,4551,0.0000
4,00087baf5cef7a.jpg,humpback_whale,8721,0.0000
...,...,...,...,...
51028,fff639a7a78b3f.jpg,beluga,5520,4.0000
51029,fff8b32daff17e.jpg,cuviers_beaked_whale,1096,1.0000
51030,fff94675cc1aef.jpg,blue_whale,5116,4.0000
51031,fffbc5dd642d8c.jpg,beluga,3909,4.0000


In [37]:
if not MODEL_TRAINED:
    model = timm.create_model(CONFIG["model_name"], pretrained=False, num_classes=CONFIG["num_classes"])
    weights = torch.load("src/pytorch_classifier/weights/Loss7.1429_epoch5.bin", map_location=torch.device("cpu"))
    model.load_state_dict(weights)
    model.to(CONFIG["device"])
    print(CONFIG["device"])


cuda:0


In [50]:
df_test = pd.read_csv("data/sample_submission.csv").drop(columns="predictions")
EXAMPLE_IMAGE_PATH = str(DATA_DIR / "test_images" / df_test.iloc[10]["image"])
EXAMPLE_IMAGE = np.array(Image.open(EXAMPLE_IMAGE_PATH).convert("RGB"))

In [39]:
df_test.head()

,image
0,000110707af0ba.jpg
1,0006287ec424cb.jpg
2,000809ecb2ccad.jpg
3,00098d1376dab2.jpg
4,000b8d89c738bd.jpg


In [49]:
@torch.inference_mode()
def generate_predictions_pt(model: nn.Module) -> pd.DataFrame:
    df = pd.read_csv("data/sample_submission.csv").drop(columns="predictions")
    test_dataset = WhaleDataset(df, transforms["valid"], labels=False)
    test_loader = DataLoader(dataset=test_dataset, num_workers=4, batch_size=32)

    with open("id_encoding.pickle", "rb") as f:
        enc = pickle.load(f)

    data = []
    for batch in test_loader:
        imgs = batch["image"].to(CONFIG["device"])
        res = F.softmax(model(imgs))

        top = res.cpu().topk(5)
        for idx, val in zip(top.indices.numpy().tolist(), top.values.numpy().tolist()):
            ids = enc.inverse_transform(idx).tolist()
            data.append(dict(zip(ids, val)))

    return_vals = [{"image": df.iloc[i]["image"], "top5": json.dumps(d)} for i,d in enumerate(data)]
    return return_vals

In [38]:
def predict_new_ind(df: pd.DataFrame, softmax_thr: float):
    
    df_local = df.copy(deep=True)

    def insert_new_ind(x, thr: float):
        data = json.loads(x)
        idvs = list(data.keys())
        ret_value = []
        for idx, (idv, score) in enumerate(data.items()):
            if score > thr:
                ret_value.append(idv)
            else:
                ret_value.append("new_individual")
                ret_value += idvs[idx:len(idvs)-1]
                break

        return " ".join(ret_value)


    df_local["predictions"] = df_local["top5"].apply(lambda x: insert_new_ind(x, softmax_thr))
    return df_local.drop(columns="top5")

In [ ]:
test = pd.read_csv(DATA_DIR / "intermediary" / "top_5.csv")

In [41]:
test.iloc[:10]

,image,top5
0,000110707af0ba.jpg,"{""a2e4dcc14c5e"": 0.048154015094041824, ""fc0f7c162cc0"": 0.04443174600601196, ""348f0fca7d1b"": 0.03226540982723236, ""a8c9dfb8ac6f"": 0.03155897557735443, ""6f632b846891"": 0.029956575483083725}"
1,0006287ec424cb.jpg,"{""04a9b1cad4d9"": 0.005956506356596947, ""7726d3269f17"": 0.005036056041717529, ""76b5aad6b790"": 0.00493953563272953, ""f1e6c5118903"": 0.004193905275315046, ""400d43387c48"": 0.0038329416420310736}"
2,000809ecb2ccad.jpg,"{""322a18725969"": 0.05694717541337013, ""51081e431bca"": 0.03947136551141739, ""dba4e482f0ad"": 0.03235378861427307, ""124534ac8131"": 0.025195930153131485, ""ff26e042cd52"": 0.022467533126473427}"
3,00098d1376dab2.jpg,"{""938b7e931166"": 0.024364719167351723, ""7362d7a01d00"": 0.01294273417443037, ""10e758eb503a"": 0.012121515348553658, ""82af241cd012"": 0.010554972104728222, ""1a20c92ffe68"": 0.010136610828340054}"
4,000b8d89c738bd.jpg,"{""bca71e9c5328"": 0.003027654020115733, ""4e0d7c4225de"": 0.0020641994196921587, ""da9cc7804e3a"": 0.0014235482085496187, ""34659a010623"": 0.001211889903061092, ""0674970efe22"": 0.0011706999503076077}"
5,000e246888710c.jpg,"{""68070283a5e9"": 0.0037413849495351315, ""fecc717ffcf8"": 0.003095830325037241, ""dcc1a00f7660"": 0.0027543315663933754, ""942976a11d81"": 0.002494653221219778, ""3b5263676547"": 0.0024914336390793324}"
6,000eb6e73a31a5.jpg,"{""be330f0c495c"": 0.045786287635564804, ""f195c38bcf17"": 0.03238629549741745, ""8bc942512479"": 0.03169039264321327, ""b54c1f8df53f"": 0.024487990885972977, ""cc0e0b020a90"": 0.023175328969955444}"
7,000fe6ebfc9893.jpg,"{""64082644c693"": 0.007232420612126589, ""fecc717ffcf8"": 0.006564832292497158, ""0f72263cd384"": 0.0056589096784591675, ""a636f358b4ae"": 0.005599102936685085, ""b602f1224dee"": 0.005490310024470091}"
8,0011f7a65044e4.jpg,"{""abbeba14a290"": 0.004166478291153908, ""48503390869b"": 0.0029057348147034645, ""22df13dec9ef"": 0.002167642116546631, ""018aaba90625"": 0.001932745799422264, ""fb11f4414d62"": 0.0018401503330096602}"
9,0012ff300032e3.jpg,"{""06ef73efe924"": 0.022979486733675003, ""8274ddd12a43"": 0.019005682319402695, ""9582cc692d85"": 0.018320536240935326, ""79c94459ece0"": 0.01826922781765461, ""da65f819073c"": 0.016991324722766876}"


In [67]:
df_thr = predict_new_ind(test, 0.04).set_index("image")

In [68]:
df_thr

,predictions
image,
000110707af0ba.jpg,a2e4dcc14c5e fc0f7c162cc0 new_individual 348f0fca7d1b a8c9dfb8ac6f
0006287ec424cb.jpg,new_individual 04a9b1cad4d9 7726d3269f17 76b5aad6b790 f1e6c5118903
000809ecb2ccad.jpg,322a18725969 new_individual 51081e431bca dba4e482f0ad 124534ac8131
00098d1376dab2.jpg,new_individual 938b7e931166 7362d7a01d00 10e758eb503a 82af241cd012
000b8d89c738bd.jpg,new_individual bca71e9c5328 4e0d7c4225de da9cc7804e3a 34659a010623
...,...
fff6ff1989b5cd.jpg,new_individual 3ee0c96472b9 be2aa86d0e5b aa169045adea b5e1ee74f22b
fff8fd932b42cb.jpg,new_individual b4e86d16be40 4e0d7c4225de 6738e57058f8 da9cc7804e3a
fff96371332c16.jpg,new_individual 8deb2171580e 7717e80dcc1a 4134fe49e6ca 322a18725969


In [69]:
df_thr.to_csv(SUB_DIR / "pytorch_classifier_thr_0,04.csv")

In [51]:
df_test

,image
0,000110707af0ba.jpg
1,0006287ec424cb.jpg
2,000809ecb2ccad.jpg
3,00098d1376dab2.jpg
4,000b8d89c738bd.jpg
...,...
27951,fff6ff1989b5cd.jpg
27952,fff8fd932b42cb.jpg
27953,fff96371332c16.jpg
27954,fffc1c4d3eabc7.jpg


In [7]:
test

,image,top5
0,000110707af0ba.jpg,"{""a2e4dcc14c5e"": 0.048154015094041824, ""fc0f7c162cc0"": 0.04443174600601196, ""348f0fca7d1b"": 0.03226540982723236, ""a8c9dfb8ac6f"": 0.03155897557735443, ""6f632b846891"": 0.029956575483083725}"
1,0006287ec424cb.jpg,"{""04a9b1cad4d9"": 0.005956506356596947, ""7726d3269f17"": 0.005036056041717529, ""76b5aad6b790"": 0.00493953563272953, ""f1e6c5118903"": 0.004193905275315046, ""400d43387c48"": 0.0038329416420310736}"
2,000809ecb2ccad.jpg,"{""322a18725969"": 0.05694717541337013, ""51081e431bca"": 0.03947136551141739, ""dba4e482f0ad"": 0.03235378861427307, ""124534ac8131"": 0.025195930153131485, ""ff26e042cd52"": 0.022467533126473427}"
3,00098d1376dab2.jpg,"{""938b7e931166"": 0.024364719167351723, ""7362d7a01d00"": 0.01294273417443037, ""10e758eb503a"": 0.012121515348553658, ""82af241cd012"": 0.010554972104728222, ""1a20c92ffe68"": 0.010136610828340054}"
4,000b8d89c738bd.jpg,"{""bca71e9c5328"": 0.003027654020115733, ""4e0d7c4225de"": 0.0020641994196921587, ""da9cc7804e3a"": 0.0014235482085496187, ""34659a010623"": 0.001211889903061092, ""0674970efe22"": 0.0011706999503076077}"
...,...,...
27951,fff6ff1989b5cd.jpg,"{""3ee0c96472b9"": 0.0030315646436065435, ""be2aa86d0e5b"": 0.00296629685908556, ""aa169045adea"": 0.002550827106460929, ""b5e1ee74f22b"": 0.0023754145950078964, ""84d91f8c1f99"": 0.00235398905351758}"
27952,fff8fd932b42cb.jpg,"{""b4e86d16be40"": 0.0008207089267671108, ""4e0d7c4225de"": 0.0005901279509998858, ""6738e57058f8"": 0.0005407998105511069, ""da9cc7804e3a"": 0.000521070440299809, ""1df65ac9eb57"": 0.00050784507766366}"
27953,fff96371332c16.jpg,"{""8deb2171580e"": 0.019156092777848244, ""7717e80dcc1a"": 0.01664147526025772, ""4134fe49e6ca"": 0.016524596139788628, ""322a18725969"": 0.013660775497555733, ""51081e431bca"": 0.012567386962473392}"
27954,fffc1c4d3eabc7.jpg,"{""180c0ab04dcd"": 0.025870470330119133, ""4a67e64bd3b7"": 0.021264269948005676, ""ea8160d46f8f"": 0.01371792983263731, ""6642e34b23c8"": 0.011253273114562035, ""be208651e012"": 0.010479174554347992}"


In [50]:
test = generate_predictions_pt(model)

In [46]:
df_test.head()

,image
0,000110707af0ba.jpg
1,0006287ec424cb.jpg
2,000809ecb2ccad.jpg
3,00098d1376dab2.jpg
4,000b8d89c738bd.jpg


In [56]:
pd.DataFrame(test).set_index("image").to_csv(DATA_DIR / "intermediary" / "top_5.csv")